# Buffer Overflow Protections
- modern kernel and compilers provide various proctections from bufferoverlow attacks

### 1. ASLR - Address Space Layout Randomization (Kernel)
- loads the stack, data and code in random memory locations making it difficult to guess their addresses
### checking ASLR
    - compile and run demo-programs/aslr.c program to check
    - run gdb-peda and aslr command

#### Check ASLR
- various values:
    - 0: disabled
    - 1: enabled (excluding data segment)
    - 2: enabled (including data segment)

In [5]:
%%bash
sudo cat /proc/sys/kernel/randomize_va_space

2


#### Set ASLR - Enable/Disable

In [6]:
%%bash
echo 0 | sudo tee /proc/sys/kernel/randomize_va_space

0


#### Allow ptrace processes
**ptrace** - allows parent process to observe and control the execution of another process
- used primarily to implement breakpoint debugging and system call tracing

In [7]:
%%bash
# check the ptrace value
sudo sysctl kernel.yama.ptrace_scope

kernel.yama.ptrace_scope = 0


In [8]:
%%bash
# set the ptrace value
sudo sysctl -w kernel.yama.ptrace_scope=0

kernel.yama.ptrace_scope = 0


### 2. PIE
- Position Independent Executable
- randomizes Code segment base address
- randomizes GOT (Global Offset Table for global data-variables) and PLT (Procedure Linkage Table)
    - PLT addresses are fixed during  linking to load the libc.so library codes/functions
    - shared libraries are position independent code (PIC); they don't know where they are being loaded
- 

### 3. Executable Stack Protection (NX) also (DEP)
- DEP - Data Execution Prevention
- in newer gcc, programs must be compiled by disabling stack protector to execute code injected into stack
- check executable stack:
`$ readelf -l <filename>`
- note the line: 
GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10
RW 
- RW - Read and Write but not Executable
- RWE - Read, Write, and Executable



In [11]:
%%bash
in=./demo-programs/hello.c
out=./demo-programs/hello
gcc -g -m32 -o $out $in 

readelf -l ./demo-programs/aslr | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RW  0x10


#### enable RWX
- compile the program using -z execstack switch of gcc

In [12]:
%%bash
in=./demo-programs/hello.c
out=./demo-programs/hello
gcc -g -m32 -z execstack -o $out $in 

readelf -l $out | grep GNU_STACK

  GNU_STACK      0x000000 0x00000000 0x00000000 0x00000 0x00000 RWE 0x10


### 4. Stack Canaries
- "canary in a coal mine" to detect carbon monoxide by using a warm blooded animal like a bird
- place a small random integer just before the stack return pointer
    - if the value is overwritten/corrupted during the function call then there's a buffer overflow attempt
- use **-fno-stack-protector** flag in gcc to disable canaries